This project explores demographic differences in immediate postsecondary enrollment among Washington high school graduates. It uses public data from data.wa.gov.
The first dataset includes data related to postseconary enrollment rates. It’s broken down by:
Year (2005-2019); area (state, county, district, school); demographic (sex, race, income, etc); and institution type (2 year, 4 year, or not enrolled) For now, I will arrange the data for all students at the county level.

In [39]:
#READ IN HIGH SCHOOL GRADUATE OUTCOMES - FIRST YEAR ENROLLMENT
#https://data.wa.gov/Education/High-School-Graduate-Outcomes-First-Year-Enrollmen/vk6s-am8z

#!/usr/bin/env python

# make sure to install these packages before running:
# pip install pandas
# pip install sodapy

import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.wa.gov", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.wa.gov,
#                  MyAppToken,
#                  username="user@example.com",
#                  password="AFakePassword")

# All results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("7ma7-qs6m", limit=400000)

# Convert to pandas DataFrame
enrollment_df = pd.DataFrame.from_records(results)

In [40]:
#filter to include all students at the school district level only
enrollment_df=enrollment_df[enrollment_df['districttype'].str.contains('School Dist')]
enrollment_df=enrollment_df[enrollment_df['demotype'].str.contains('All Students')]
enrollment_df=enrollment_df[enrollment_df['cohorttype'].str.contains('1yr')]

In [41]:
#drop unused columns
enrollment_df.drop(['districttype','schoolttl','redactedpct','demotype',
                    'demographicgroup', 'demographicvalue', 'redactedpct', 'cohorttype'],
                   axis=1, inplace=True)
enrollment_df.reset_index(drop=True, inplace=True)

In [42]:
#change value 'pct' & 'cohortyearttl' from object to number
enrollment_df['pct']=enrollment_df['pct'].apply(pd.to_numeric, errors='coerce')
enrollment_df['cohortyearttl']=enrollment_df['cohortyearttl'].apply(pd.to_numeric, errors='coerce')

In [43]:
#filter to only include years >= 2014
enrollment_df=enrollment_df[enrollment_df['cohortyearttl']>=2014]

In [44]:
#pivot from long to wide data frame and reset index
df1=pd.pivot_table(enrollment_df,
                   index=['cohortyearttl','districtttl'],
                   columns='psenrolllevel',
                   values='pct').reset_index()

#remove index name
df1.index.name = df1.columns.name = None

#rename columns
df1.rename(columns={'cohortyearttl': 'Year', 'districtttl': 'District', '2 Year / CTC' : 'TwoYear', '4 Year' : 'FourYear', 'Not Enrolled' : 'NotEnrolled'}, inplace=True)


In [45]:
#check clean data frame
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1284 entries, 0 to 1283
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Year         1284 non-null   int64  
 1   District     1284 non-null   object 
 2   TwoYear      1284 non-null   float64
 3   FourYear     1284 non-null   float64
 4   NotEnrolled  1284 non-null   float64
dtypes: float64(3), int64(1), object(1)
memory usage: 50.3+ KB


The second dataset inlcudes demographic data. It includes raw counts of students broken down by:

Year (2014-15 - 2021-22);
area (state, county, district, school);
demographic

In [46]:
#LOAD DISTRICT DEMOGRAPHIC DATA
#https://data.wa.gov/education/Report-Card-Enrollment-from-2014-15-to-Current-Yea/rxjk-6ieq

#!/usr/bin/env python

# make sure to install these packages before running:
# pip install pandas
# pip install sodapy

import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.wa.gov", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.wa.gov,
#                  MyAppToken,
#                  username="user@example.com",
#                  password="AFakePassword")

# All results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
demo = client.get("rxjk-6ieq", limit=200000)

# Convert to pandas DataFrame
demo_df = pd.DataFrame.from_records(demo)

In [47]:
#filter to include all students at the school district level only
demo_df=demo_df[demo_df['organizationlevel'].str.contains('District')]
demo_df=demo_df[demo_df['gradelevel'].str.contains('AllGrades')]

In [48]:
#drop unused columns
demo_df.drop(['organizationlevel','county','esdname','schoolname',
              'gradelevel', 'dataasof', 'esdorganizationid', 'districtcode',
              'districtorganizationid', 'schoolcode', 'schoolorganizationid', 'currentschooltype',
              'fostercare', 'non_fostercare', 'students_without_disabilities','students_with_disabilities',
              'section_504', 'non_section_504',
              'mobile', 'non_mobile', 'military_parent', 'non_military_parent','migrant', 'non_migrant',
              'homeless', 'non_homeless',
              ],
             axis=1, inplace=True)
demo_df.reset_index(drop=True, inplace=True)

In [49]:
#drop 'School District' after district name to match with df1
demo_df['districtname'] = demo_df['districtname'].str.replace(' School District', '')

#drop multi-year title from school year to match df1, i.e. 2014-15 == 2014
demo_df['schoolyear'] = demo_df['schoolyear'].str[:-3]

In [50]:
#change most columns from characters to numeric
severalToNum=lambda x:pd.to_numeric(x,errors='coerce')
where=demo_df.columns[2:]
demo_df.loc[:,where]=demo_df.loc[:,where].apply(severalToNum)
demo_df['schoolyear']=demo_df['schoolyear'].apply(pd.to_numeric, errors = "coerce")

#change year column from character to numeric
demo_df = demo_df[demo_df['schoolyear']<=2019]

/var/folders/7s/92wdsp7x3jb14hws7v5952t40000gn/T/ipykernel_36088/3780803281.py:4: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  demo_df.loc[:,where]=demo_df.loc[:,where].apply(severalToNum)


In [51]:
#rename columns & reset index
df2=demo_df
df2.rename(columns={'schoolyear': 'Year', 'districtname': 'District'}, inplace=True)
df2.reset_index(drop=True, inplace=True)

In [52]:
#create rate variables
df2['FemalePct'] = df2['female']/df2['all_students']
df2['WhitePct'] = df2['white']/df2['all_students']
df2['ELLPct'] = df2['english_language_learners']/df2['all_students']
df2['LowIncomePct'] = df2['low_income']/df2['all_students']
df2['HighlyCapablePct'] = df2['highly_capable']/df2['all_students']
#rename all_students variable
df2.rename(columns={'all_students': 'TotalStudents'}, inplace=True)

In [53]:
#remove unused variables
df2.drop(df2.iloc[:, 3:19], axis=1, inplace=True)
df2.reset_index(drop=True, inplace=True)

In [54]:
#check clean data frame
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1928 entries, 0 to 1927
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Year              1928 non-null   int64  
 1   District          1928 non-null   object 
 2   TotalStudents     1928 non-null   int64  
 3   FemalePct         1928 non-null   float64
 4   WhitePct          1928 non-null   float64
 5   ELLPct            1928 non-null   float64
 6   LowIncomePct      1928 non-null   float64
 7   HighlyCapablePct  1928 non-null   float64
dtypes: float64(5), int64(2), object(1)
memory usage: 120.6+ KB


In [55]:
#merge clean dataframes
df3 = pd.merge(df1, df2, on=['District', 'Year'])

#check merged data
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 927 entries, 0 to 926
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Year              927 non-null    int64  
 1   District          927 non-null    object 
 2   TwoYear           927 non-null    float64
 3   FourYear          927 non-null    float64
 4   NotEnrolled       927 non-null    float64
 5   TotalStudents     927 non-null    int64  
 6   FemalePct         927 non-null    float64
 7   WhitePct          927 non-null    float64
 8   ELLPct            927 non-null    float64
 9   LowIncomePct      927 non-null    float64
 10  HighlyCapablePct  927 non-null    float64
dtypes: float64(8), int64(2), object(1)
memory usage: 86.9+ KB


In [56]:
#print merged data
df3

,Year,District,TwoYear,FourYear,NotEnrolled,TotalStudents,FemalePct,WhitePct,ELLPct,LowIncomePct,HighlyCapablePct
0,2014,Aberdeen,0.4250,0.180,0.395,3404,0.487955,0.573443,0.098120,0.726204,0.028202
1,2014,Adna,0.4200,0.120,0.470,610,0.470492,0.903279,0.000000,0.288525,0.000000
2,2014,Anacortes,0.2750,0.425,0.305,2708,0.494092,0.839734,0.022526,0.331610,0.016987
3,2014,Arlington,0.3000,0.275,0.425,5575,0.477309,0.787444,0.038924,0.374709,0.000538
4,2014,Auburn,0.2825,0.325,0.390,15685,0.492828,0.451450,0.149251,0.596047,0.014472
...,...,...,...,...,...,...,...,...,...,...,...
922,2019,Asotin-Anatone,0.2000,0.420,0.370,625,0.502400,0.862400,0.000000,0.379200,0.051200
923,2019,Lake Washington Institute of Technology,0.2900,0.240,0.470,806,0.423077,0.648883,0.000000,0.000000,0.000000
924,2019,College Place,0.5200,0.190,0.290,1629,0.476980,0.495396,0.176796,0.574586,0.062615
925,2019,Summit Public School: Olympus,0.1500,0.480,0.360,183,0.497268,0.267760,0.060109,0.715847,0.081967


In [57]:
#write merged data to csv
df3.to_csv('waenrolldf.csv', index=False)